# Algoritmos de Interpolación Cúbica, Barzilai Borwein y Zhan Hager 

## Interpolación cúbica

In [130]:
#Se ocupa el algoritmo de interpolación cuadrática para calcular el alpha1
#aqui estamos usando dirección de descenso el -gradiente
import numpy as np
from numpy import linalg as la
import math
def backtracking_direction(x,a,p,c,dd,tf,delta): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd)> tf(x)+c*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def inter_cuadratica(a0,x_k,tf,grad,d_k,c1): #dado un punto a0, el x_k la función y la tolerancia del gradiente vamos a hacer lo siguiente
    if tf(x_k+a0*d_k)>tf(x_k)+c1*a0*np.dot(grad(x_k).T,d_k): 
        numer=-a0**2*np.dot(grad(x_k).T,d_k)
        denom=2*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
        a1=numer*(1/denom)
        while tf(x_k+a1*d_k)>tf(x_k)+c1*a1*np.dot(grad(x_k).T,d_k): #no satisface la condición de armijo
            a0=a1
            numer=-a0**2*np.dot(grad(x_k).T,d_k)
            denom=2*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
            a1=numer*(1/denom)
    else:
        a1=a0
    return a1

def inter_cubica(a0,x_k,tf,grad,d_k,c1):
    a1=inter_cuadratica(a0,x_k,tf,grad,d_k,c1)
    #No se satisface armijo para ninguna de las dos condiciones de armijo
    if tf(x_k+a1*d_k)>tf(x_k)+c1*a1*np.dot(grad(x_k).T,d_k) and tf(x_k+a0*d_k)>tf(x_k)+c1*a0*np.dot(grad(x_k).T,d_k):
        d=tf(x_k)
        c=np.dot(grad(x_k).T,d_k)
        vec=np.zeros(2)
        vec[0]=a0**2*(tf(x_k+a1*d_k)-a1*np.dot(grad(x_k).T,d_k)-tf(x_k))-a1**2*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
        vec[1]=-a0**3*(tf(x_k+a1*d_k)-a1*np.dot(grad(x_k).T,d_k)-tf(x_k))+a1**3*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
        vec=(1/((a1**2*a0**2)*(a1-a0)))*vec
        #print(vectoresab)
        a2=(-1*vec[1]+math.sqrt(vec[1]**2-3*vec[0]*c))/(3*vec[0])
        #print(vectoresab)
        #print(vec)
        while tf(x_k+a2*d_k)>tf(x_k)+c1*a2*np.dot(grad(x_k).T,d_k):
            a0=a1
            a1=a2
            d=tf(x_k)
            c=np.dot(grad(x_k).T,d_k)
            vec[0]=a0**2*(tf(x_k+a1*d_k)-a1*np.dot(grad(x_k).T,d_k)-tf(x_k))-a1**2*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
            vec[1]=-a0**3*(tf(x_k+a1*d_k)-a1*np.dot(grad(x_k).T,d_k)-tf(x_k))+a1**3*(tf(x_k+a0*d_k)-a0*np.dot(grad(x_k).T,d_k)-tf(x_k))
            vec=(1/((a1**2*a0**2)*(a1-a0)))*vec
            #print(vec)
            #print(vectoresab)
            a2=(-1*vec[1]+math.sqrt(vec[1]**2-3*vec[0]*c))/(3*vec[0])
    else:
        a2=a1
    return a2


def newton_line_search(x0,mxitr,tolgrad,tf,gf,H): #todas las condiciones de las funciones de búsqueda en linea.
    iterations=0
    g_k=gf(x0)
    g_k_norm=la.norm(g_k)
    g=-1*np.dot(la.inv(H(x0)),g_k)
    #print(g)
    difrelx=1e10 #inicializamos las diferencias
    difrelf=1e10
    puntoiter=x0
    flag=True
    stepsize=inter_cubica(1,puntoiter,tf,gf,g,1e-4)
    #stepsize=1e-2 #intentemos con paso fijo
    while iterations<mxitr and g_k_norm>tolgrad:
        g_k=gf(puntoiter)
        g_k_norm=la.norm(g_k)
        while flag==True:
            try:
                la.cholesky(H(puntoiter))
                flag=False#checo si la matriz es definida positiva que garantiza la dirección de descenso
            except:
                print('Matriz no definida positiva lo que no grantiza descenso')
                H(puntoiter)+np.identity(H(puntoiter).shape[0])
        stepsize=inter_cubica(1,puntoiter,tf,gf,g,1e-4)
        g=-1*np.dot(la.inv(H(puntoiter)),g_k)#esta es la dirección de descenso garantizada porque H es positiva definida 
        puntoiter=puntoiter+stepsize*g
        iterations=iterations+1
        #print(iterations-1, '&', la.norm(g_k),'&',tf(puntoiter) )
    return np.array([iterations,la.norm(g_k),tf(puntoiter)]) #el punto que es y lo que vale 


In [131]:
#funcion de prueba a wood de n=2
 #Definimos las funciones que vamos a utilizar en este caso
    #usaremos el método de newton para probar este caso
def f(datos): #Función objetivo o target function
    x,y=datos
    return 100*(y-x**2)**2+(1-x)**2

def gradiente_f(datos): #gradiente de la función f
    g=np.zeros(len(datos))
    g[0]=2*(200*datos[0]**3-200*datos[0]*datos[1]+datos[0]-1)
    g[1]=200*(datos[1]-datos[0]**2)
    return g

def hessiano_f(datos):
    hes=np.zeros((len(datos),len(datos)))
    hes[0][0]=2*(600*datos[0]**2-200*datos[1]+1)
    hes[1][0]=-400*datos[0]
    hes[0][1]=-400*datos[0]
    hes[1][1]=200
    return hes


In [132]:
res=newton_line_search([-1.2,1],3000,1e-4,f,gradiente_f,hessiano_f)
print(res)

[1.00000000e+02 1.70929704e-09 1.69739061e-20]


# Barzilai Borwein

In [133]:
def backtracking_direction(x,a,p,c,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd)> tf(x)+c*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Barzilai_Borwein_descenso(x0,maxitr,epsilon,grad,tf): #la función en este caso no se ocupa
    k=0
    g_k=grad(x0)
    g=-1*g_k
    x_k=x0
    iterations=0
    while la.norm(g_k)>epsilon and iterations<maxitr:
        if k==0:
            a=backtracking_direction(x_k,1,0.5,0.5,g,tf)
        else:
            a=np.dot((x_k-x_kant).T,(g_k-g_k_t))*(1/np.dot((g_k-g_k_t).T,(g_k-g_k_t)))
        x_kant=x_k
        g_k_t=grad(x_kant)
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        k=k+1
        iterations=iterations+1
    return np.array([iterations,la.norm(g_k),tf(x_k)])

In [134]:
res=Barzilai_Borwein_descenso([-1.2,1],2000,1e-4,gradiente_f,f)
print(res)

[1.90000000e+01 1.37029279e-06 2.34378508e-12]


# Zhang Hager

In [135]:
def modified_backtracking_direction(x,C,a,p,c1,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd) > C +c1*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Zhang_Hager(x0,maxitr,epsilon,nu,grad,tf):
    k=0
    g_k=grad(x0)
    g=-1*g_k
    iterations=0
    Q=1
    x_k=x0
    C=tf(x_k)
    trial=modified_backtracking_direction(x_k,C,1,0.5,0.5,g,tf)
    while la.norm(g_k)>epsilon and iterations<maxitr:
        a=modified_backtracking_direction(x_k,C,trial,0.5,0.5,g,tf)
        trial=a
        Q_ant=Q
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        Q=Q_ant+1
        C=(nu*Q_ant*C+tf(x_k))/Q
        iterations=iterations+1
    return np.array([iterations,la.norm(g_k),tf(x_k)])

In [136]:
res=Zhang_Hager([-1.2,1],2000,1e-4,1.0,gradiente_f,f)
print(res)

[2.00000000e+03 9.56432111e-02 9.73723591e-03]


# Caso de prueba 1: Funcion Rosenbruck n=100

In [137]:
#Definamos la función que vamos a usar 
def rosen(x): #La función de rosembruck en general
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

def r_der(x):
        xm = x[1:-1]
        xm_m1 = x[:-2]
        xm_p1 = x[2:]
        der =np.zeros(x.shape)
        der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
        der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
        der[-1] = 200*(x[-1]-x[-2]**2)
        return der

def rosen_hess(x):
        H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
        diagonal = np.zeros(len(x))
        diagonal[0] = 1200*x[0]-400*x[1]+2
        diagonal[-1] = 200
        diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
        H = H + np.diag(diagonal)
        return H

In [138]:
#Vector de prueba del algoritmo
test=np.ones(100)
test[0]=1.2
test[98]=-1.2


In [139]:
res=newton_line_search(test,2000,1e-4,rosen,r_der,rosen_hess)
print(res)

[2.40000000e+01 1.90124877e-05 5.39760782e-11]


In [141]:
res=Barzilai_Borwein_descenso(test,2000,1e-4,r_der,rosen)
print(res)

[8.85000000e+02 9.87641780e-05 7.30916061e-09]


In [144]:
res=Zhang_Hager(test,200000,1e-4,1.0,r_der,rosen)
print(res)

[7.19750000e+04 9.99927783e-05 1.00220659e-08]


# Caso 2: Funcion Wood

In [145]:
def wood(datos): #Función general, target function
    x1,x2,x3,x4=datos
    return 100*((x2-x1**2)**2)+(1-x1)**2+(1-x3)**2+90*((x4-x3**2)**2)+10.1*((x2-1)**2+(x4-1)**2)+19.8*(x2-1)*(x4-1)

def wood_derivate(x):
    g=np.zeros(len(x))
    g[0]=2.0*(200.0*x[0]**3-200*x[0]*x[1]+x[0]-1)
    g[1]=-200*x[0]**2+220.2*x[1]+19.8*x[3]-40
    g[2]=2*(180*x[2]**3-180*x[3]*x[2]+x[2]-1)
    g[3]=19.8*x[1]-180*x[2]**2+200.2*x[3]-40
    return g

def wood_hessian(x):
    hes=np.zeros((len(x),len(x)))
    hes[0][0]=400*(x[0]**2-x[1])+800*x[0]**2+2
    hes[0][1]=-400*x[0]
    hes[1][0]=400*x[0]
    hes[1][1]=220.2
    hes[1][3]=19.8
    hes[2][2]=2+720*x[2]**2+360*(x[2]**2-x[3])
    hes[2][3]=-360*x[2]
    hes[3][1]=19.8
    hes[3][2]=-360*x[2]
    hes[3][3]=200.2
    return hes
    

In [146]:
res=newton_line_search(np.array([-3,-1,-3,-1]),2000,1e-4,wood,wood_derivate,wood_hessian)
print(res)

[2.00000000e+03 4.97870664e-03 6.98602452e-06]


In [147]:
res=Barzilai_Borwein_descenso(np.array([-3,-1,-3,-1]),2000,1e-4,wood_derivate,wood)
print(res)

[1.01000000e+02 2.08468685e-05 7.23780008e-13]


In [148]:
res=Zhang_Hager(np.array([-3,-1,-3,-1]),20000,1e-4,1.0,wood_derivate,wood)
print(res)

[2.00000000e+04 4.23805020e-03 1.24803929e-05]
